In [2]:
#!pip install gym

  Using cached gym-0.17.2.tar.gz (1.6 MB)
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Using cached cloudpickle-1.3.0-py2.py3-none-any.whl (26 kB)
  Created wheel for gym: filename=gym-0.17.2-py3-none-any.whl size=1650896 sha256=ec84a0af0f57751f74799eafbf6d4d46e84124e65afdb526faa5a127307b9c97
  Stored in directory: c:\users\mosan\appdata\local\pip\cache\wheels\be\a1\84\6b4caa6c1cea703acbfea8a24cc3c1729bd359cd4a65755d8b
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.5.0
    Uninstalling cloudpickle-1.5.0:
      Successfully uninstalled cloudpickle-1.5.0


In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import gym

env = gym.make('CartPole-v0')

In [3]:
observation = env.reset() #observation return ..
print(observation)

[ 0.02999755 -0.00842166  0.03755834 -0.03345345]


In [4]:
for _ in range(2): # episode 2번 생성

    env.reset()

    for _ in range(50):
        env.render()
        action = env.action_space.sample() # action 랜덤으로 가져옴
        observation, reward, done, info = env.step(action) # Transition 일으킴

        print(observation, reward, done, info)

        if done:
            break

[ 0.03828893  0.24252556  0.03166984 -0.26770723] 1.0 False {}
[ 0.04313944  0.43718155  0.0263157  -0.55023543] 1.0 False {}
[ 0.05188307  0.24170005  0.01531099 -0.24937878] 1.0 False {}
[ 0.05671707  0.43660004  0.01032341 -0.53719324] 1.0 False {}
[ 6.54490743e-02  6.31575331e-01 -4.20451560e-04 -8.26605567e-01] 1.0 False {}
[ 0.07808058  0.43645913 -0.01695256 -0.53405491] 1.0 False {}
[ 0.08680976  0.24157964 -0.02763366 -0.24676149] 1.0 False {}
[ 0.09164136  0.43708513 -0.03256889 -0.54803103] 1.0 False {}
[ 0.10038306  0.63264913 -0.04352951 -0.85079501] 1.0 False {}
[ 0.11303604  0.82833672 -0.06054541 -1.15684199] 1.0 False {}
[ 0.12960278  0.634054   -0.08368225 -0.88374137] 1.0 False {}
[ 0.14228386  0.44016201 -0.10135708 -0.618495  ] 1.0 False {}
[ 0.1510871   0.63654276 -0.11372698 -0.9413023 ] 1.0 False {}
[ 0.16381795  0.83299848 -0.13255302 -1.26744593] 1.0 False {}
[ 0.18047792  1.02954091 -0.15790194 -1.59852995] 1.0 False {}
[ 0.20106874  0.83660311 -0.18987254 -1

In [5]:
env.action_space.sample()
env.step?

In [6]:
print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

Box(4,)
Discrete(2)
0


In [7]:
print(env.observation_space.low)
print(env.observation_space.high)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [8]:
n_observation = env.observation_space.shape
n_actions = env.action_space.n

print(n_observation)
print(n_actions)

(4,)
2


In [9]:
n_bins_pos = 10
n_bins_vel = 10
n_bins_ang = 10
n_bins_anv = 10

In [11]:
n_states = n_bins_pos*n_bins_vel*n_bins_ang*n_bins_anv
n_action = 2

Q_table = np.random.uniform(0, 1,(n_states, n_actions)) # 0~1 값 & shape

In [12]:
print(Q_table, Q_table.shape)

[[0.4720598  0.03861795]
 [0.61408808 0.19113143]
 [0.63307965 0.9249879 ]
 ...
 [0.07325044 0.18650188]
 [0.97000806 0.6947951 ]
 [0.64647507 0.04529677]] (10000, 2)


In [13]:
env.reset()
observation, _, _, _ = env.step(0)
env.close()

pos, vel, ang, anv = observation
print(observation)

[-0.02587327 -0.1790236   0.01811598  0.27319424]


In [14]:
def map_discrete_state(state):
    pos, vel, ang, anv = state
    
    idx_pos = np.where(np.histogram(np.clip(pos, -2, 2), bins = n_bins_pos, range=(-2,2))[0]==1)[0][0]
    idx_vel = np.where(np.histogram(np.clip(vel, -2, 2), bins = n_bins_vel, range=(-2,2))[0]==1)[0][0]
    idx_ang = np.where(np.histogram(np.clip(ang, -0.4, 0.4), bins = n_bins_ang, range=(-0.4,0.4))[0]==1)[0][0]
    idx_anv = np.where(np.histogram(np.clip(anv, -2, 2), bins = n_bins_anv, range=(-2,2))[0]==1)[0][0]
    
    states = np.zeros([n_bins_pos, n_bins_vel, n_bins_ang, n_bins_anv])
    states[idx_pos, idx_vel, idx_ang, idx_anv] = 1
    
    states = states.reshape(-1, 1) # 일렬로 세움
    s = np.where(states == 1)[0][0]
    
    return s


In [15]:
np.where(np.histogram(np.clip(pos, -2, 2), bins = n_bins_pos, range=(-2,2))[0]==1)[0][0]

4

In [16]:
alpha = 0.3
gamma = 0.9

Q_table = np.random.uniform(0, 1, (n_states, n_actions))

for episode in range(801):
    
    done = False
    
    state = env.reset()
    
    count = 0
    while not done:
        count += 1
        
        s = map_discrete_state(state)
        
        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q_table[s, :])
        
        next_state, reward, done, _ = env.step(a)
        
        if done:
            reward = -100
            Q_table[s, a] = reward
        else:
            next_s = map_discrete_state(next_state)
            Q_table[s, a] = (1-alpha)*Q_table[s,a] + (alpha)*(reward + gamma*np.argmax(Q_table[next_s, :]))        
        
        state = next_state
    
    if episode % 100 == 0:
        print(episode, count)
env.close()

0 18
100 56
200 12
300 17
400 33
500 36
600 17
700 20
800 11


In [27]:
state = env.reset()

done = False

while not done:
    env.render()
    
    s = map_discrete_state(state)
    a = np.argmax(Q_table[s, :])
    
    next_state, _, done, _ = env.step(a)
    
    state = next_state
    
env.close()